### Import Packages

In [1]:
import pandas as pd
import requests
import sqlite3
import random
import time
import json

In [2]:
user_agents = open('user_agent_list.txt', 'r').read().split('\n')

In [3]:
def get_request(request_url):
    response = requests.get(
        url=request_url,
        headers={'user-agent':  random.choice(user_agents)}
    )
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return "Error: " + str(e)
    return response

In [4]:
# Define wine type ids as defined by vivino.com

wine_type_ids = {
    'red': 1,
    'white': 2,
    'sparkling_wine': 3,
    'rose': 4,
    'liqueur_wine': 24,
    'dessert_wine': 7
}

### Collecting the wines

In [46]:
con = sqlite3.connect('wine.db')
cur = con.cursor()
cur.execute('CREATE TABLE wine('
            'id integer primary key , name, price, type_id, is_natural, winemaker, region, country, '
            'grapes, average_rating, has_valid_ratings)')

In [41]:
def save_wine_data(wine_json):
    wine_data = []
    for i in range(len(wine_json['explore_vintage']['matches'])):
        grapes = []
        try:
            for grape_index in range(len(wine_json['explore_vintage']['matches'][i]['vintage']['wine']['style']['grapes'])):
                grapes.append(wine_json['explore_vintage']['matches'][i]['vintage']['wine']['style']['grapes'][grape_index]['name'])
            grapes = ', '.join(grapes)
        except:
            grapes = None
        wine_data.append(
            (
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['id'],
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['name'],
                wine_json['explore_vintage']['matches'][i]['price']['amount'],
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['type_id'],
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['is_natural'],
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['winery']['name'],
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['region']['name'],
                wine_json['explore_vintage']['matches'][i]['vintage']['wine']['region']['country']['name'],
                grapes,
                wine_json['explore_vintage']['matches'][i]['vintage']['statistics']['wine_ratings_average'],
                wine_json['explore_vintage']['matches'][i]['vintage']['has_valid_ratings']
            )
        )
    cur.executemany('INSERT INTO wine VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', wine_data)
    con.commit()
    con.close()

In [47]:
for wine_type in wine_type_ids.values():
    for i in range(5):
        url = 'https://www.vivino.com/api/explore/explore?wine_type_ids[]={}&page={}'.format(wine_type, i)
        response = get_request(request_url=url)
        save_wine_data(response.json())
        time.sleep(random.randint(2, 10))

In [2]:
con = sqlite3.connect('wine.db')
cur = con.cursor()

In [3]:
df = pd.read_sql_query("SELECT * from wine", con)

In [56]:
df = df.groupby(by='id').agg({'name': 'first', 'price': 'mean', 'type_id': 'first', 'is_natural': 'first',
                         'winemaker': 'first', 'region': 'first', 'country': 'first', 'grapes': 'first', 'average_rating': 'first',
                         'has_valid_ratings': 'first'}).reset_index()

In [57]:
df.to_sql('wine',con=con, if_exists='replace')

322